In [53]:
"""

From Dima Pasechnik, for the n=2 case:

Here is a complete algorithm for \( n = 2 \). Let 
\[
H = \begin{pmatrix} \alpha & \omega \\ \overline{\omega} & \beta \end{pmatrix},
\]
with \( \alpha = \overline{\alpha} \) and \( \beta = \overline{\beta} \). We look for 
\[
L = \begin{pmatrix} 1 & 0 \\ a & 1 \end{pmatrix}
\]
such that \( LHL^* \) is diagonal. If \( \omega = 0 \), we are done (since \( H \) is diagonal, and we can factor \( \alpha = \tau \overline{\tau} \) and \( \beta = \mu \overline{\mu} \) to get \( H = I \)).

We have
\[
LHL^* = \begin{pmatrix} \alpha & \alpha \overline{a} + \omega \\ a \alpha + \overline{\omega} & a \overline{a} \alpha + a \omega + \overline{a} \overline{\omega} + \beta \end{pmatrix}.
\]

If \( \alpha \neq 0 \), we can set \( a := -\frac{\overline{\omega}}{\alpha} \) and obtain a diagonal result.

If \( \beta \neq 0 \), we can pre-conjugate \( H \) by the permutation matrix \( \Pi \) of \( (1, 2) \), yielding a nonzero \( \alpha \), and proceed as above. Thus, in this case, the conjugation is by \( L\Pi \).

If \( \alpha = \beta = 0 \), in \( LHL^* \) we obtain a new \( \beta := a\omega + \overline{a}\overline{\omega} \). We then apply conjugation by \( \Pi \) to achieve a nonzero \( \alpha \), and proceed as above to find a suitable \( L' \). In this case, the conjugation is by \( L'\Pi L \).


""";

In [23]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [24]:
def cholesky_finite_field(A):
    """
    Perform Cholesky decomposition of a Hermitian matrix A over a finite field with a conjugate structure
    (e.g., GF(q^2) where each element has a well-defined conjugate operation).
    
    Parameters:
        A (Matrix): A Hermitian (conjugate symmetric) matrix over a finite field with a conjugate operation.
    
    Returns:
        L (Matrix): A lower triangular matrix such that A = L * conjugate_pos_char(L).transpose(), if decomposition is possible.

    Issue:
        Not returning correct matrix. Only getting A = LL^T.
    """
    n = A.nrows()
    L = Matrix(A.base_ring(), n, n, 0)  # Initialize an n x n matrix of zeros in the same field as A
    field_size = A.base_ring().order()
    q = int(sqrt(field_size)) if field_size.is_square() else field_size

    for i in range(n):
        # Calculate the diagonal element L[i, i]
        diag_sum = sum(L[i, k] * L[i, k]**q for k in range(i))
        diag_val = A[i, i] - diag_sum
        
        # Check if diag_val has a square root in the field
        if not diag_val.is_square():
            raise ValueError(f"No square root exists for the value at diagonal {i}: {diag_val}")
        
        # Assign the square root of diag_val to L[i, i]
        L[i, i] = diag_val.sqrt()
        
        # Calculate the off-diagonal elements L[j, i] for j > i
        for j in range(i + 1, n):
            off_diag_sum = sum(L[j, k] * L[i, k]**q for k in range(i))
            L[j, i] = (A[j, i] - off_diag_sum) / L[i, i]

    assert A == L * conjugate_pos_char(L).transpose()
    
    return L

In [54]:
def cholesky_brute_force(U,q):
    """
        Compute the Cholesky decomposition U = LL^* by brute force, where * denotes conjugate transpose and we are using the conjugation x |--> x^q.

        This will only work for small values of q, n. We can look at representations of dimension d_rho = 2, 3, so that d_rho^2 = 4, 9. 
        
        We work over GF(q^2) where we have square roots, for q=2, 3. If n=3,4, then p|n is problematic, but we can try q=5. 

        For example, for q=3, q^2=9, n=4, d_rho = 3, d_rho^2 = 9, we would have 9^9 = 387420489 possibilities.
    """
    from itertools import product

    # Define the finite field GF(3^2)
    F = GF(q^2)
    elements = list(F) # List of all elements in GF(3^2)

    # Define the size of the matrix
    n = U.nrows()
    num_entries = n + (n * (n - 1)) // 2
    num_iters = 0
    
    # Iterate over all possible combinations of elements
    for a in product(elements, repeat=num_entries):
        num_iters += 1
        # Construct the lower triangular matrix
        if n == 2:
            L = matrix(F,[[a[0],0],[a[1],a[2]]])
        if n ==3:
            L = matrix(F, [[a[0], 0, 0],[a[1], a[2], 0],[a[3], a[4], a[5]]])
    
        if U == L * conjugate_pos_char(L).transpose():
            return L
    
        if num_iters % 10^4 == 0:
            print(float(num_iters / (q^2)^6))

In [16]:
import time

# Start the timer
start_time = time.time()

# Your loop or code block
for i in range(10^9):
    pass  # Replace with your code

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Elapsed time: 23.6245539188385 seconds


In [43]:
U_0 = matrix(GF(3^2),[[1,2],[2,1]]); U_0

[1 2]
[2 1]

In [34]:
U_1 = matrix(GF(3^2),[[1,2,2],[2,1,2],[2,2,1]]); U_1

[1 2 2]
[2 1 2]
[2 2 1]

In [38]:
U_2 = matrix(GF(3^2),[[0,1,2],[1,0,1],[2,1,0]]); U_2

[0 1 2]
[1 0 1]
[2 1 0]

In [50]:
cholesky_brute_force(U_0,q)

[  z2 + 1        0]
[2*z2 + 2        0]